In [ ]:
import pandas as pd
from lib.preprocessing import *
from lib.cleaning import *
import seaborn as sns


# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer
# from mpl_toolkits.basemap import Basemap
# import matplotlib.pyplot as plt
# %matplotlib inline
# import missingno as msno

### Exploration

In [ ]:
df = load_sample()

In [ ]:
cols = pd.Series(df.columns.format())

metrics = ["Temperature", "Precipitation in millimeters", "Distance (KM)"]  # 3 of these

ids = pd.concat([cols.loc[cols.str.match(".*Id.*")], cols.loc[cols.str.match(".*No.*")]]) # 3 of these

location = ["Pickup Lat", "Pickup Long", "Destination Lat", "Destination Long"] # 4 of these
times = cols.loc[cols.str.match(".*Time.*")] # 6(actually 4) of these

types = ["Vehicle Type", "Platform Type", "Personal or Business", ] # 3 of these


In [ ]:
times

In [ ]:
ids

In [ ]:
desc = pd.read_csv("variable_definitions.csv")
desc

### Cleaning Data

#### Nelson cleaned data

In [ ]:
df = load_sample(percentage=1, csv_file="data/train_full.csv")
df.head(3)

In [ ]:
df = load_sample(percentage=1, csv_file="data/train_full.csv")
df = drops(df)
df = impute_temperature(df)
df = combine_weekdays(df)
df.head(3)

In [ ]:
df.loc[(df['Placement - Day of Month'] < df["Fulfillment - Day of Month"])]

In [ ]:
save_data(df, sql_db="data/cleaned_nelson.db")

#### Merging Emily w/ Nelson

In [ ]:
em = load_sample(percentage=1, sql_db="data/cleaned_emily.db")

In [ ]:
em = em.drop(columns=["level_0", "index"]).rename({
    "Order_No" : "order_no",
    "User_Id" : "user_id",
    "Rider_Id" : "rider_id"
})

In [ ]:
em.head(3)

In [ ]:
save_data(em, sql_db="data/cleaned_emily_nelson.db")

#### Merging Alex w/ Emily+Nelson

In [ ]:
riders_clean = load_sample(percentage=1, sql_db="data/cleaned_Alex.db")
# riders_clean.rename({"id" : "Rider_Id"}, axis=1, inplace=True)
riders_clean.drop(columns="index", inplace=True)
riders_clean["id"] = riders_clean["id"].astype(int)
riders_clean.set_index("id", inplace=True)
riders_clean.head(3)
# riders_clean.dtypes

In [ ]:
other_clean = load_sample(percentage=1, sql_db="data/cleaned_emily_nelson.db")
other_clean.head(3)

In [ ]:
cleaned_all = other_clean.join(riders_clean, on="Rider_Id", rsuffix="_rider").drop(columns="index")

In [ ]:
save_data(cleaned_all, sql_db="data/cleaned_alex_emily_nelson.db")

### Model testing